## 수치 미분을 이용한 심층 신경망 학습 (꼭! 이해 필요)

In [1]:
import time
import numpy as np

## 유틸리티 함수

In [2]:
# 미분을 구하기 위한 학습률(Learing Rate)
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

# 활성함수(activation Function)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

## 뉴런 구현

In [3]:
class Neuron:
    def __init__(self, W, b, a):
        # Model Parameter
        self.W = W # Matrix : 가중치
        self.b = b # Vector : 편향
        self.a = a # Vector : 활성함수
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

    def __call__(self, x):
        # _t() transpose 함수를 사용자 함수로 재구현
        # _m() matmul 함수를 사용자 함수로 재구현
        return self.a(_m(_t(self.W), x) + self.b) # activation((W^T)x + b)

## 심층신경망 구현

In [4]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            # Weight과 bias 값을 초기화 해서 넘겨줌
            # i는 input 갯수, o는 output 갯수
            # 평균 0, 표준편차가 0.01인 값을 i x o 배열로 초기화함
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))  # Matrix(Weight)와 Vector(bias)를 리턴

        self.sequence = list()
        # First hidden layer의 weight와 bias를 초기화
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))
        
        # Hidden layers의 weight와 bias를 초기화
        for _ in range(hidden_depth - 1):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output layer의 weight와 bias를 초기화
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Neuron(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func): # x는 입력, y는 정답, loss_func은 손실함수
        
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def eval_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        # 기준이 되는 loss값을 계산
        loss = loss_func(self(x), y) # self(x)는 self.__call__(x)와 동일
        
        for layer_id, layer in enumerate(self.sequence): # layer를 처음부터 끝까지 순회 Hidden Layer * input Node * output Node
            for w_i, w in enumerate(layer.W): # input Node * oupput Node Matrix 들을 순회
                for w_j, ww in enumerate(w): # Weight Vector 들을 순회
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon # 기울기를 얻기위해 weight값을 변경
                    
                    # weight 값만 변경된 Layer 생성
                    new_neuron = Neuron(W, layer.b, layer.a) # W는 기존의 Weight Matrix, layer.b는 기존의 bias Vector, layer.a는 기존의 activation 함수
                    
                    # Weigth 값이 변경된 Layer로 생성된 새로운 Network를 만듬
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    
                    # 입력값 x를 새로운 Network에 적용
                    h = eval_sequence(x, new_seq)
                    
                    # numerial gradient를 계산함
                    num_grad = (loss_func(h, y) - loss) / epsilon  # (f(x + epsilon) - f(x)) / epsilon
                    
                    # 변경된 weight값으로 생성된 gradient를 저장
                    layer.dW[w_i][w_j] = num_grad
            
                for b_i, bb in enumerate(layer.b): # bias Vector만큼 반복 수행
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon # 기울기를 얻기위해 bias값을 변경
                    
                    # bias 값만 변경된 Layer 생성
                    new_neuron = Neuron(layer.W, b, layer.a)
                    
                    # bias 값이 변경된 Layer로 생성된 새로운 Network를 만듬
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    
                    # 입력값 x를 새로운 Network에 적용
                    h = eval_sequence(x, new_seq)
                    
                    #numerial gradient를 계산함
                    num_grad = (loss_func(h, y) - loss) / epsilon  # (f(x + epsilon) - f(x)) / epsilon

                    # 변경된 bias 값으로 생성된 gradient를 저장
                    layer.db[b_i] = num_grad
        return loss

## 경사하강 학습법

In [5]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    
    # loss값을 구하여, weight와 bias값을 변경하여 저장하고, 변경된 weight와 bias를 이용하여 새로운 loss값을 구하고
    # 새로운 loss값을 이용하여 weight와 bias의 gradient(기울기)를 구하여 저장하고
    # 초기 loss값을 돌려줌
    loss = network.calc_gradient(x, y, loss_obj)
    
    idx = 1
    # weight와 bias에 학습율(alpha)과 기울기를 이용하여 다시 계산함(왜?)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW # layer.W = layer.W - (alpha * layer.dW)
        layer.b += -alpha * layer.db # layer.b = layer.b - (alpha * layer.db)
        if idx == 1:
            print('layer.W[0] => ', layer.W[0])
            print('layer.dW[0] => ', layer.dW[0])
            print('layer.b => ', layer.b)
            print('layer.db => ', layer.db)
            idx = idx + 1
    return loss

## 동작 테스트

In [6]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

print(x)
print(y)

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

[ 0.48267527 -0.38775626  1.52108748  1.77850943 -0.91266055  0.17654513
 -0.3027063  -0.00777281  0.28752181 -0.52113981]
[ 0.61065483 -0.50888061]
layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 3.33066907e-12  8.88178420e-12  1.66533454e-12 -2.22044605e-12
 -7.21644966e-12  2.22044605e-12 -9.99200722e-12 -2.22044605e-12
  5.55111512e-13 -6.66133815e-12 -2.22044605e-12 -2.22044605e-12
 -1.66533454e-12  4.44089210e-12  2.22044605e-12  4.99600361e-12
  0.00000000e+00 -1.11022302e-11 -7.77156117e-12 -1.66533454e-12
 -5.55111512e-12 -5.55111512e-12 -7.77156117e-12  2.22044605e-12
  1.11022302e-12 -1.27675648e-11 -6.106

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 4.44089210e-12  1.11022302e-11  4.99600361e-12 -1.11022302e-12
 -3.33066907e-12  4.44089210e-12 -8.88178420e-12 -3.88578059e-12
  1.66533454e-12 -5.55111512e-12 -1.11022302e-12 -2.77555756e-12
 -1.11022302e-12  4.44089210e-12  3.33066907e-12  7.21644966e-12
 -1.66533454e-12 -1.11022302e-11 -6.66133815e-12  0.00000000e+00
 -2.77555756e-12 -3.88578059e-12 -7.77156117e-12  3.33066907e-12
  2.22044605e-12 -1.05471187e-11 -6.10622664e-12 -5.55111512e-12
  5.55111512e-12 -7.21644966e-12 -6.10622664e-12 -3.88578059e-12]
layer.b =>  [-4.44089210e-13 -9.93649607e-13 -3.83026943e

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 4.71844785e-12  1.27675648e-11  6.66133815e-12 -5.55111512e-13
 -1.94289029e-12  4.99600361e-12 -8.88178420e-12 -6.10622664e-12
  2.22044605e-12 -6.10622664e-12 -5.55111512e-13 -5.27355937e-12
 -2.22044605e-12  3.60822483e-12  1.66533454e-12  7.49400542e-12
 -4.16333634e-12 -1.36002321e-11 -8.32667268e-12  2.77555756e-13
 -2.49800181e-12 -3.88578059e-12 -9.43689571e-12  3.60822483e-12
  2.77555756e-12 -1.05471187e-11 -8.88178420e-12 -5.55111512e-12
  3.05311332e-12 -7.77156117e-12 -6.66133815e-12 -6.93889390e-12]
layer.b =>  [-8.29891711e-13 -1.97342143e-12 -8.99280650e

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 5.27355937e-12  1.41553436e-11  8.88178420e-12  0.00000000e+00
 -2.77555756e-13  6.66133815e-12 -8.60422844e-12 -8.60422844e-12
  2.22044605e-12 -6.10622664e-12 -1.11022302e-12 -6.93889390e-12
 -2.77555756e-12  2.49800181e-12  1.38777878e-12  8.04911693e-12
 -5.82867088e-12 -1.60982339e-11 -8.60422844e-12  1.38777878e-12
 -1.94289029e-12 -2.77555756e-12 -1.02695630e-11  4.16333634e-12
  2.77555756e-12 -9.43689571e-12 -9.71445147e-12 -5.55111512e-12
  2.49800181e-12 -6.93889390e-12 -6.10622664e-12 -8.32667268e-12]
layer.b =>  [-1.23512311e-12 -3.07809334e-12 -1.58206781e

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 5.27355937e-12  1.52655666e-11  1.08246745e-11  0.00000000e+00
  1.94289029e-12  7.77156117e-12 -7.49400542e-12 -1.08246745e-11
  2.22044605e-12 -5.82867088e-12 -1.11022302e-12 -8.88178420e-12
 -3.33066907e-12  1.94289029e-12  1.11022302e-12  9.15933995e-12
 -7.77156117e-12 -1.74860126e-11 -8.88178420e-12  2.22044605e-12
 -1.94289029e-12 -2.22044605e-12 -1.13797860e-11  5.27355937e-12
  2.77555756e-12 -9.15933995e-12 -1.13797860e-11 -4.44089210e-12
  1.11022302e-12 -7.21644966e-12 -4.99600361e-12 -9.43689571e-12]
layer.b =>  [-1.67088565e-12 -4.31599201e-12 -2.42583731e

...

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 4.16333634e-12  3.08086889e-11  2.77555756e-11  7.21644966e-12
  1.72084569e-11  1.80411242e-11 -4.99600361e-12 -3.35842465e-11
  7.49400542e-12 -7.21644966e-12  6.10622664e-12 -2.33146835e-11
 -1.27675648e-11 -3.88578059e-12 -3.05311332e-12  2.05391260e-11
 -2.55351296e-11 -3.44169138e-11 -1.02695630e-11  1.02695630e-11
  5.55111512e-12  1.94289029e-12 -2.02615702e-11  1.05471187e-11
  3.60822483e-12 -1.94289029e-12 -2.19269047e-11 -2.77555756e-13
 -1.13797860e-11 -8.04911693e-12  2.22044605e-12 -2.41473508e-11]
layer.b =>  [-9.04831765e-12 -3.68372000e-11 -3.05172554e

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 3.88578059e-12  3.13638004e-11  2.77555756e-11  6.93889390e-12
  1.63757896e-11  1.77635684e-11 -5.27355937e-12 -3.41393580e-11
  7.21644966e-12 -8.04911693e-12  6.66133815e-12 -2.47024623e-11
 -1.41553436e-11 -4.44089210e-12 -3.60822483e-12  2.08166817e-11
 -2.58126853e-11 -3.55271368e-11 -1.02695630e-11  9.99200722e-12
  4.99600361e-12  1.66533454e-12 -2.08166817e-11  1.02695630e-11
  3.05311332e-12 -2.22044605e-12 -2.27595720e-11 -5.55111512e-13
 -1.24900090e-11 -9.43689571e-12  1.94289029e-12 -2.52575738e-11]
layer.b =>  [-9.39526235e-12 -3.94045907e-11 -3.28098659e

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 3.60822483e-12  3.16413562e-11  2.77555756e-11  7.21644966e-12
  1.69309011e-11  1.83186799e-11 -5.55111512e-12 -3.46944695e-11
  7.77156117e-12 -8.04911693e-12  6.66133815e-12 -2.52575738e-11
 -1.41553436e-11 -4.71844785e-12 -3.05311332e-12  2.10942375e-11
 -2.58126853e-11 -3.52495810e-11 -1.02695630e-11  1.05471187e-11
  5.55111512e-12  1.11022302e-12 -2.10942375e-11  1.05471187e-11
  2.77555756e-12 -2.22044605e-12 -2.27595720e-11  0.00000000e+00
 -1.24900090e-11 -9.43689571e-12  1.94289029e-12 -2.52575738e-11]
layer.b =>  [-9.72832925e-12 -4.20163904e-11 -3.51413343e

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 3.88578059e-12  3.16413562e-11  2.83106871e-11  7.49400542e-12
  1.72084569e-11  1.83186799e-11 -4.99600361e-12 -3.58046925e-11
  8.04911693e-12 -8.88178420e-12  7.49400542e-12 -2.49800181e-11
 -1.49880108e-11 -4.71844785e-12 -3.05311332e-12  2.13717932e-11
 -2.60902411e-11 -3.55271368e-11 -9.71445147e-12  1.13797860e-11
  4.99600361e-12  1.11022302e-12 -2.13717932e-11  1.05471187e-11
  3.05311332e-12 -1.66533454e-12 -2.27595720e-11  0.00000000e+00
 -1.30451205e-11 -9.99200722e-12  2.49800181e-12 -2.58126853e-11]
layer.b =>  [-1.00364161e-11 -4.46670478e-11 -3.75033338e

layer.W[0] =>  [-0.00120099 -0.01056514 -0.00295554 -0.00465818  0.01860106  0.01859521
 -0.00845661 -0.00172741  0.01572617  0.00275652  0.00807633  0.01006611
  0.01316328  0.00356397 -0.00375006 -0.00878835 -0.00227353 -0.00058585
  0.0046062  -0.00428397  0.00156397 -0.00046777 -0.00688428  0.00308896
 -0.01108046 -0.00071352 -0.01306103  0.00895229  0.02111028 -0.00582636
  0.01709634  0.00179356]
layer.dW[0] =>  [ 3.05311332e-12  3.24740235e-11  2.91433544e-11  7.21644966e-12
  1.69309011e-11  1.88737914e-11 -5.27355937e-12 -3.66373598e-11
  7.77156117e-12 -8.60422844e-12  8.04911693e-12 -2.55351296e-11
 -1.55431223e-11 -4.99600361e-12 -3.05311332e-12  2.22044605e-11
 -2.69229083e-11 -3.58046925e-11 -9.99200722e-12  1.11022302e-11
  5.82867088e-12  1.11022302e-12 -2.19269047e-11  1.08246745e-11
  2.77555756e-12 -1.38777878e-12 -2.27595720e-11  0.00000000e+00
 -1.27675648e-11 -9.99200722e-12  2.49800181e-12 -2.55351296e-11]
layer.b =>  [-1.03167475e-11 -4.73537876e-11 -3.98792110e